In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./tests/sample_inputs/sample_input_1.csv')
df

,id,gender,race,decision,reward,state1,state2,state3
0,1,0,1,1,2,1,4,6
1,1,0,1,2,4,5,7,3
2,1,0,1,2,3,6,7,1
3,2,1,0,3,4,5,6,7
4,2,1,0,2,4,2,7,3
5,2,1,0,1,4,8,10,1
6,3,1,3,3,1,6,7,8
7,3,1,3,1,2,4,5,6
8,3,1,3,2,9,4,3,6


In [4]:
df['gender'].to_numpy()

array([0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int64)

In [5]:
df[['state1', 'state2', 'state3']].to_numpy()

array([[ 1,  4,  6],
       [ 5,  7,  3],
       [ 6,  7,  1],
       [ 5,  6,  7],
       [ 2,  7,  3],
       [ 8, 10,  1],
       [ 6,  7,  8],
       [ 4,  5,  6],
       [ 4,  3,  6]], dtype=int64)

In [6]:
df[['state3', 'state1']]

,state3,state1
0,6,1
1,3,5
2,1,6
3,7,5
4,3,2
5,1,8
6,8,6
7,6,4
8,6,4


In [8]:
df['state1']

0    1
1    5
2    6
3    5
4    2
5    8
6    6
7    4
8    4
Name: state1, dtype: int64

In [ ]:
df[['decision', 'reward']].to_numpy()[[1, 2, 4, 5, 7, 8], :]

array([[2, 4],
       [2, 3],
       [2, 4],
       [1, 4],
       [1, 2],
       [2, 9]], dtype=int64)

In [ ]:
def f_x0_multi(zs, ux0, z_coef=1):
    gamma0 = np.array([np.repeat(np.array([0.5]), repeats=3), 
                       np.repeat(np.array(1 * z_coef), repeats=3), 
                       np.repeat(np.array(1 * z_coef), repeats=3)])
    n = zs.shape[0]
    M = np.concatenate(
        [
            np.ones([n, 1]),
            zs,
        ],
        axis=1,
    )
    x0 = M @ gamma0
    x0 = x0.reshape(-1, 3)
    x0 = x0 + ux0
    return x0

# xt_i = -0.5 + zs_1 + zs_2 + 3 * (xtm1_1 + xtm1_2 + xtm1_3) + 2 * atm1 + uxt (assuming z_coef=1)
def f_xt_multi(zs, xtm1, atm1, uxt, z_coef=1):
    gamma = np.array([np.repeat(np.array([-0.5]), repeats=3), 
                      np.repeat(np.array(1 * z_coef), repeats=3), 
                      np.repeat(np.array(1 * z_coef), repeats=3), 
                      np.array([3, 3, 3]),
                      np.array([3, 3, 3]), 
                      np.array([3, 3, 3]), 
                      np.array([2, 2, 2])])
    n = xtm1.shape[0]
    M = np.concatenate(
        [
            np.ones([n, 1]),
            zs,
            xtm1,
            atm1.reshape(-1, 1), 
        ],
        axis=1,
    )
    xt = M @ gamma
    xt = xt.reshape(-1, 3)
    xt = xt + uxt
    return xt

# xt = -0.5 + zs_1 + zs_2 + xt_1 + xt_2 + xt_3 + at + urt (assuming z_coef=1)
def f_rt_multi(zs, xt, at, urtm1, z_coef=1):
    lmbda = np.array([-0.5, 1, 1, 1, 1 * z_coef, 1 * z_coef, 1, 1])
    n = xt.shape[0]
    at = at.reshape(-1, 1)
    M = np.concatenate(
        [np.ones([n, 1]), xt, zs, at, urtm1], axis=1
    )
    rt = M @ lmbda
    return rt

In [9]:
np.random.multivariate_normal(mean=[0, 0, 0], cov=np.diag([1, 1, 1]), size=[3, 1])

array([[[ 0.44024102, -1.39814094, -1.73934431]],

       [[ 1.20021192,  0.92412868, -1.53677974]],

       [[ 1.00114166,  1.7021257 , -1.86427798]]])

In [11]:
np.random.normal(0, 1, [2, 2])

array([[-0.01168136, -1.26451211],
       [ 0.28766569,  1.20187007]])

In [13]:
zs = np.array([[1, 1], [2, 2]])
ux0 = np.random.multivariate_normal(mean=[0, 0, 0], cov=np.diag([1, 1, 1]), size=2)
x0 = f_x0_multi(zs, ux0)
x0

array([[0.54220223, 1.59587365, 2.90970319],
       [4.95905874, 4.33671607, 4.09413163]])

In [15]:
action = np.array([0, 1])
ux1 = np.random.multivariate_normal(mean=[0, 0, 0], cov=np.diag([1, 1, 1]), size=2)
ur0 = np.random.normal(0, 1, size=[2, 1])
x1 = f_xt_multi(zs, x0, action, ux1)
x1

array([[16.97615704, 15.61275672, 16.07228417],
       [46.43271941, 46.58292937, 44.56175276]])

In [19]:
r0 = f_rt_multi(zs, x0, action, ur0)
r0

array([ 6.16966841, 18.15582582])